# HW3 - Web crawling

HDS - Napong Leelasithorn - 61070505228 <br><br>
• Starting with a seed page, www.cpe.kmutt.ac.th <br>
• Build a web spider and crawler to download all links of 
CPE website and prepare them as DataFrames <br><br>
• Deliverables <br>
1. Web Spider DataFrame, must at least contain 2 columns <br>
• from the original URL contains that link the first time <br>
• to the extracted URL
2. Content DataFrame. Must at least contain 3 columns <br>
• no running number (index) of this table <br>
• url the URL (only those from the CPE website) <br>
• content text content

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import regex as re
import pandas as pd
from selenium.webdriver.edge.options import Options

In [2]:
driver = webdriver.Edge(executable_path = 'msedgedriver.exe')

## links/urls finding function

In [3]:
def urlfinder(url):
    
    """input: url
       output: return 2 lists [from], [to]
       the function will find both links from <a> and <button>, filter only
       valid websites or paths, then return the modified list.
       *this function will works only with cpe.kmutt.ac.th website,
       otherwise, it returns 2x input url-lists*
    """
    
    if r'https://cpe.kmutt.ac.th' in str(url):
        driver.get(url)
        sleep(0.07)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        buttons = soup.select('button')
        ref1 = []
        for i in buttons:
            try:
                ref1.append(re.search(r'https?://.+', i.get('onclick'))[0][:-1])
    
            except:
                pass
            
        a = soup.select('a')
        tmpref = [j.get('href') for j in a]
        for href in tmpref:
            try: 
                ref1.append(re.search(r'(https?://.+|\/th\/.+|\/en\/.+)',
                                      href)[0] \
                            if re.match(r'https?://.+',href) \
                            else 'https://cpe.kmutt.ac.th'+\
                            re.search(r'(https?://.+|\/th\/.+|\/en\/.+)',
                                      href)[0])
            except:
                pass
        return  [url]*len(ref1), ref1
    
    else:
        return [url],[url]

## web spider

In [4]:
allfrom = []
allto = []
languages = ['/th/index', '/en/index']
url = 'https://cpe.kmutt.ac.th'
for language in languages:
    urllan = url + language
    from_, to = urlfinder(urllan)
    allfrom += from_
    allto += to
for newurl in allto:
    if newurl in allfrom:
        pass
    else:
        from_, to = urlfinder(newurl)
        allfrom += from_
        allto += to

# construct dataframe and drop duplicates
df = pd.DataFrame({'from_website': allfrom,
              'to_website': allto}).drop_duplicates()
# select only non self path
deriverable1 = df[df['from_website'] != df['to_website']]
deriverable1

,from_website,to_website
0,https://cpe.kmutt.ac.th/th/index,http://cpe.kmutt.ac.th/th/post/347
1,https://cpe.kmutt.ac.th/th/index,https://tcas.cpe.kmutt.ac.th/
3,https://cpe.kmutt.ac.th/th/index,https://cpe.kmutt.ac.th/th/department
4,https://cpe.kmutt.ac.th/th/index,https://cpe.kmutt.ac.th/th/staff
5,https://cpe.kmutt.ac.th/th/index,https://cpe.kmutt.ac.th/th/student
...,...,...
1596,https://cpe.kmutt.ac.th/th/index.php,http://www.kmuttalumni.com/
1597,https://cpe.kmutt.ac.th/th/index.php,http://www.lib.kmutt.ac.th/
1598,https://cpe.kmutt.ac.th/th/index.php,http://www.cc.kmutt.ac.th/
1599,https://cpe.kmutt.ac.th/th/index.php,https://www.facebook.com/cpe.kmutt


## exclude index.php

In [5]:
deriverable1 = deriverable1[~(deriverable1['from_website'].str.contains('index.php')) & 
                            ~(deriverable1['to_website'].str.contains('index.php'))]
deriverable1

,from_website,to_website
0,https://cpe.kmutt.ac.th/th/index,http://cpe.kmutt.ac.th/th/post/347
1,https://cpe.kmutt.ac.th/th/index,https://tcas.cpe.kmutt.ac.th/
3,https://cpe.kmutt.ac.th/th/index,https://cpe.kmutt.ac.th/th/department
4,https://cpe.kmutt.ac.th/th/index,https://cpe.kmutt.ac.th/th/staff
5,https://cpe.kmutt.ac.th/th/index,https://cpe.kmutt.ac.th/th/student
...,...,...
1554,https://cpe.kmutt.ac.th/en/document,http://coop.kmutt.ac.th/
1555,https://cpe.kmutt.ac.th/en/document,http://www.kmuttalumni.com/
1556,https://cpe.kmutt.ac.th/en/document,http://www.lib.kmutt.ac.th/
1557,https://cpe.kmutt.ac.th/en/document,http://www.cc.kmutt.ac.th/


## exclude "-" from staff website (2 after droped duplicates)

## 1. Web Spider DataFrame <br>
Web Spider DataFrame, must at least contain 2 columns
1. from the original URL contains that link the first time
2. to the extracted URL

In [6]:
deriverable1 = deriverable1[~(deriverable1['to_website'] == 'http://-')]
deriverable1

,from_website,to_website
0,https://cpe.kmutt.ac.th/th/index,http://cpe.kmutt.ac.th/th/post/347
1,https://cpe.kmutt.ac.th/th/index,https://tcas.cpe.kmutt.ac.th/
3,https://cpe.kmutt.ac.th/th/index,https://cpe.kmutt.ac.th/th/department
4,https://cpe.kmutt.ac.th/th/index,https://cpe.kmutt.ac.th/th/staff
5,https://cpe.kmutt.ac.th/th/index,https://cpe.kmutt.ac.th/th/student
...,...,...
1554,https://cpe.kmutt.ac.th/en/document,http://coop.kmutt.ac.th/
1555,https://cpe.kmutt.ac.th/en/document,http://www.kmuttalumni.com/
1556,https://cpe.kmutt.ac.th/en/document,http://www.lib.kmutt.ac.th/
1557,https://cpe.kmutt.ac.th/en/document,http://www.cc.kmutt.ac.th/


from this dataframe, we're able to find furthur knowledge by construct a graph network, then we'll see the relationship between each node(site) more clearly.

### -----------------------------------------------------------------------------------------------------------------------------------------------------------

## get all unique links

In [7]:
uniq_url = pd.concat([deriverable1['from_website'], deriverable1['to_website']], axis = 0).drop_duplicates()
uniq_url

0                        https://cpe.kmutt.ac.th/th/index
40                       https://cpe.kmutt.ac.th/en/index
85                  https://cpe.kmutt.ac.th/th/department
121                      https://cpe.kmutt.ac.th/th/staff
184                    https://cpe.kmutt.ac.th/th/student
                              ...                        
1331    http://global.kmutt.ac.th/admissions/internati...
1332    http://admission.kmutt.ac.th/bachelor/page-adm...
1333    http://global.kmutt.ac.th/admissions/undergrad...
1334                http://global.kmutt.ac.th/admissions/
1336    http://global.kmutt.ac.th/admissions/graduate/...
Length: 77, dtype: object

## get page source

In [8]:
pgsrc = []
for link in uniq_url:
    if r'https://cpe.kmutt.ac.th' in link:
        driver.get(link)
        sleep(0.05)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        pgsrc.append(soup)
    else:
        pgsrc.append('-')

In [9]:
tmp = pd.DataFrame({'url': uniq_url,
                    'page_source': pgsrc})
tmp

,url,page_source
0,https://cpe.kmutt.ac.th/th/index,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
40,https://cpe.kmutt.ac.th/en/index,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
85,https://cpe.kmutt.ac.th/th/department,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
121,https://cpe.kmutt.ac.th/th/staff,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
184,https://cpe.kmutt.ac.th/th/student,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
...,...,...
1331,http://global.kmutt.ac.th/admissions/internati...,-
1332,http://admission.kmutt.ac.th/bachelor/page-adm...,-
1333,http://global.kmutt.ac.th/admissions/undergrad...,-
1334,http://global.kmutt.ac.th/admissions/,-


## 2. Content DataFrame<br>
Must at least contain 3 columns
1. no running number (index) of this table
2. url the URL (only those from the CPE website)
3. content text content

In [10]:
deriverable2 = tmp[tmp['page_source'] != '-'].reset_index().drop('index', axis = 1)
deriverable2

,url,page_source
0,https://cpe.kmutt.ac.th/th/index,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
1,https://cpe.kmutt.ac.th/en/index,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
2,https://cpe.kmutt.ac.th/th/department,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
3,https://cpe.kmutt.ac.th/th/staff,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
4,https://cpe.kmutt.ac.th/th/student,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
5,https://cpe.kmutt.ac.th/th/achievement,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
6,https://cpe.kmutt.ac.th/th/research,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
7,https://cpe.kmutt.ac.th/th/internationalise,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
8,https://cpe.kmutt.ac.th/th/undergrad-regular,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."
9,https://cpe.kmutt.ac.th/th/undergrad-inter,"[[[\n, <meta charset=""utf-8""/>, \n, <meta cont..."


In [11]:
len(deriverable2)

40